In [ ]:
# テキストコーパスをチャンクに分割
with open('code_gate.py', 'r', encoding='utf-8') as f:
    text = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

# コードに特化したスプリッターを使用（Pythonコードの場合）
text_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON,
    chunk_size=500,  # コード理解に適したサイズ
    chunk_overlap=50  # 適度なオーバーラップで文脈を保持
)
texts = text_splitter.split_text(text)

In [ ]:
# パッセージのベクトル化
from langchain_huggingface import HuggingFaceEmbeddings

# コードに特化したエンベディングモデルを使用
embeddings = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',  # 多言語対応で高性能なエンベディングモデル
    model_kwargs={'device': 'cpu'}
)

In [ ]:
from langchain_community.vectorstores import FAISS

# データベースの保存
db = FAISS.from_texts(texts, embeddings)
db.save_local('code.db')

In [ ]:
from langchain_community.vectorstores import FAISS

# 保存したデータベースの読み込み
db = FAISS.load_local('code.db', embeddings, allow_dangerous_deserialization=True)

In [ ]:
# Retrieverの設定 - 適切なパラメータで検索精度を向上
retriever = db.as_retriever(
    search_type="mmr",  # Maximum Marginal Relevanceで多様性のある検索結果を得る
    search_kwargs={
        "k": 5,  # 取得するドキュメント数
        "fetch_k": 20,  # MMR探索用の初期取得数
        "lambda_mult": 0.7  # 関連性と多様性のバランス (0.0-1.0)
    }
)

In [ ]:
# モデルの準備
import os
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = 'sk-proj-*****'

openai_llm = ChatOpenAI(model_name="gpt-4.1")
#openai_llm = ChatOpenAI(model_name="gpt-4.1-nano")
qa = RetrievalQA.from_chain_type(
    llm=openai_llm,
    retriever=retriever,
    return_source_documents=True,
    )

In [ ]:
# 実行例
q = "どんなプログラミング言語で書かれていますか？"
ans = qa.invoke(q)
print(q)
print(ans['result'])

print("--------------------------")

q = "定義されている関数を全てリストアップして下さい"
ans = qa.invoke(q)
print(q)
print(ans['result'])

print("--------------------------")

q = "AND関数は何回呼ばれますか？"
ans = qa.invoke(q)
print(q)
print(ans['result'])

print("--------------------------")

q = "このプログラムがやっていることは何ですか？"
ans = qa.invoke(q)
print(q)
print(ans['result'])